In [70]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [71]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
#chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe',options=chrome_options)

In [72]:
url='https://www.ntca.org/events-education/events/events-calendar'

driver.get(url)
time.sleep(5)

soup = bs(driver.page_source,'lxml')
cards=soup.find_all('div',{'class':'flip-card'})


In [77]:
ntca=[]
for m in cards:
    tit=' '.join(m.h3.text.split())
    info=' '.join(m.p.text.split())
    try:
        datiloc=' '.join(m.find('div',{'class':'absolute'}).text.split()).split(':')[1].strip()
    except:
        datiloc=[a for a in [' '.join(a.text.split()) for a in m.find('p').next_siblings] if len(a)>1][0].split(':')[1].strip()
    link=m.find('a')['href']
    tiff=m.find('div',{'class':'flip-card-back'}).find_all('li')
    rat=[' '.join(a.text.split()) for a in tiff]
    rat.insert(0,datiloc)
    #while 'TBA' in dat: dat.remove('TBA')
    while True:
        try:
            rat.remove('TBA')
        except ValueError:
            break
            
            
    for i in rat:
        dat=re.search('[A-Za-z]{3,9}\s?\d+\W?\d+\W?\s+\d*|[A-Za-z]+\s?\d+\s?\W?\s?[A-Za-z]+\s?\d+\W?\s?\d*',i).group()
        #******************************
        dat=dat.replace('–','-').replace('|','').replace('to','-').replace('TO','-').replace('c-ber','ctober').replace('C-BER','CTOBER').strip()
        if '(' in dat:
            dat=dat.split('(')[0].strip()
        
        if dat=='':
            start_date=end_date=''
        else:
            if not re.search('\d{4}',dat):
                dat=dat+' '+str(date.today().year)
            mad=dat
            #try:
    #             ma=re.search('([A-Sa-z]+\W+\d{1,2}.+\W+\d{4}|\d{1,2}.+\W+\d{4})',_date)#, maxsplit=0)
    #             mad=ma.group()

            if '-' in mad:
                st=mad.split('-')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
                en=mad.split('-')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

            elif 'and'in mad:
                st=mad.split('and')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
                en=mad.split('and')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

            elif '&' in mad:
                st=mad.split('&')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
                en=mad.split('&')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

            else:
                st=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
                en=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
            #print(date__,st,en)
            ####################
            if any(c.isalpha() for c in st)==False:
            #      print('leg')
                pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
                sapa=pa.group()
                start=st+' '+sapa
                end=en
            #                        02        April      2022
            elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
             #   print('awujale')
                start=st
                end=en

            #*                        April       02      2022
            elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
            #       print('ala')
                pa=re.search('(\d{4})',st).group()
                sa=re.search('([A-Sa-z]{3,9})',st).group()
                ta=re.search('(\d{1,2})',st).group()
                start=ta+' '+sa+' '+pa
                #
                ba=re.search('(\d{4})',en).group()
                ca=re.search('([A-Sa-z]{3,9})',en).group()
                da=re.search('(\d{1,2})',en).group()
                end=da+' '+ca+' '+ba     

            #                      02     2022
            elif re.search('\d{1,2}\s+\d{4}',en):
            #      print('kum')
                pa=re.search('(\d{4})',en).group()#, maxsplit=0)
                sa=re.search('([A-Sa-z]{3,9})',st).group()
                ta=re.search('(\d{1,2})',st).group()

                start=ta+' '+sa+' '+pa
                #
                ba=re.search('(\d{4})',en).group()
                ca=re.search('([A-Sa-z]{3,9})',st).group()
                da=re.search('(\d{1,2})',en).group()
                end=da+' '+ca+' '+ba

            #                      02         April
            elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
            #        print('is')
                pa=re.search('(\d{4})',en)#, maxsplit=0)
                sapa=pa.group()
                start=st+' '+sapa
                end=en

            #*                      April           02
            elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
            #      print('bad')
                pa=re.search('(\d{4})',en).group()#, maxsplit=0)
                sa=re.search('([A-Sa-z]{3,9})',st).group()
                ta=re.search('(\d{1,2})',st).group()

                start=ta+' '+sa+' '+pa
                #
                ba=re.search('(\d{4})',en).group()
                ca=re.search('([A-Sa-z]{3,9})',en).group()
                da=re.search('(\d{1,2})',en).group()
                end=da+' '+ca+' '+ba

            else:
            #     print('shik')
                start=end=''
            if start=='':
                start_date=end_date=''
            else:
                pick=[start, end]
                try:
                    spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
                except:
                    spl_dt_obj = [datetime.strptime(v, '%d %b %Y') for v in pick]
                date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
                start_date=date_[0]
                end_date=date_[1]
        
        
        #******************************
        link=link
        title=tit
        event_info=info
        locca=i.replace(dat,'').replace('—','').strip()
        if re.search('\d+.+\d{4}',locca):
            locca=re.sub('[A-Za-z]+\s?\d+\W*[A-Za-z]+\s?\d+\W?\s?\d{4}','',locca).replace('  ',' ').replace('  ',' ')
        if locca.startswith('|'):
            loc=locca.replace('|','').strip()
            city=loc
        else:
            city=locca.split('|')[-1].strip()
            loc=locca.replace(' | ',', ')
        
        #****************************************************
        #     #**************
        def gu(luc):
            google_url_for_location="https://www.google.com/search?q="+luc+"&oq="+luc+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            try:
                google_map_url=driver.find_element_by_id('lu_map').click()
            except:
                try:
                    google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                except:
                    google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
            time.sleep(1)
            google_map_url=driver.current_url
    #                 print(google_map_url)
            return(google_map_url)
            ######################################
        def get_google_map_url(location):
            try:
                return(gu(location))
            except:
                try:
                    return(gu(location))
                except:
                    sha=location.split(',')
                    try:
                        gu(sha[-3])
                    except:
                        try:
                            gu(sha[-2])
                        except:
                            try:
                                gu(sha[-1])
                            except Exception as e:
                                print(location, "; url didn't go through")
                                print(e)
                                return("")


        def countr(locale):
            try:
                google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
                time.sleep(randint(0,3))
                driver.get(google_url_for_location)
                time.sleep(4)
                try:
                    google_map_url=driver.find_element_by_id('lu_map').click()
                except:
                    try:
                        google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                    except:
                        try:
                            google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
                        except:
                            google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
                            time.sleep(randint(0,3))
                            driver.get(google_url_for_location)
                            time.sleep(4)
                            google_map_url=driver.find_element_by_id('lu_map').click()
                time.sleep(1)
                try:
                    country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
                except:
                    country=driver.find_element_by_class_name('bwoZTb').text
    #                 print(country)
                return(country)
            except Exception as e:
                print(locale, ";country didn't go through")
                print(e)
                return("")

        time_=''
        org_name='NTCA–The Rural Broadband Association'
        org_web='https://www.ntca.org/'
        org_pro='NTCA–The Rural Broadband Association is building a better broadband future for rural America. Proudly representing nearly 850 independent, family-owned and community-based telecommunications companies, NTCA’s members build and deliver broadband connectivity and operate essential services in rural and small-town communities across the U.S.'
        logo=''    
        sponsor=''
        agendalist=''
        type_=''
        category=''
        Speakerlist='' 
        ticket_list=''
        event_web=link
        mail_=json.dumps(['meetings@ntca.org'])

        if 'nline' in loc or 'ff Campus' in loc or loc=='' or loc==' ' or loc=='-' or loc=='Virtual' or 'nspecified' in loc or 'irtual' in loc:
            on_off='1'
            city=''
            venue=''
            country=''
            googlePlaceUrl=''
        else:
            on_off='0'
            
            country=countr(city)
            venue=loc
            googlePlaceUrl=get_google_map_url(venue)

        
        ntca.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                     org_pro,org_name,org_web,logo,sponsor,agendalist,
                     type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                     Speakerlist,on_off])

Charlotte Marriott City Center, Charlotte, NC ;country didn't go through
Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="lu_map"]"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x00AF6463+2188387]
	Ordinal0 [0x00A8E461+1762401]
	Ordinal0 [0x009A3D78+802168]
	Ordinal0 [0x009D1880+989312]
	Ordinal0 [0x009D1B1B+989979]
	Ordinal0 [0x009FE912+1173778]
	Ordinal0 [0x009EC824+1099812]
	Ordinal0 [0x009FCC22+1166370]
	Ordinal0 [0x009EC5F6+1099254]
	Ordinal0 [0x009C6BE0+945120]
	Ordinal0 [0x009C7AD6+948950]
	GetHandleVerifier [0x00D971F2+2712546]
	GetHandleVerifier [0x00D8886D+2652765]
	GetHandleVerifier [0x00B8002A+520730]
	GetHandleVerifier [0x00B7EE06+516086]
	Ordinal0 [0x00A9468B+1787531]
	Ordinal0 [0x00A98E88+1805960]
	Ordinal0 [0x00A98F75+1806197]
	Ordinal0 [0x00AA1DF1+1842673]
	BaseThreadInitThunk [0x75E7FA29+25]
	RtlGetAppContainerNamedObjectPath [0x773F7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x773F7A6E+238]


In [78]:
len(ntca)

25

In [79]:
ntca

[['https://www.ntca.org/events-education/events/rural-telecom-industry-meeting-expo-rtime',
  'Rural Telecom Industry Meeting + EXPO (RTIME)',
  '2023-02-19',
  '2023-02-22',
  '',
  'RTIME is America’s largest annual rural telecommunications industry event that delivers a universal education on policy, technology, healthcare, agriculture, community growth and advocacy.',
  '',
  'NTCA–The Rural Broadband Association is building a better broadband future for rural America. Proudly representing nearly 850 independent, family-owned and community-based telecommunications companies, NTCA’s members build and deliver broadband connectivity and operate essential services in rural and small-town communities across the U.S.',
  'NTCA–The Rural Broadband Association',
  'https://www.ntca.org/',
  '',
  '',
  '',
  '',
  '',
  'San Diego, CA',
  'California',
  'February 19–22, 2023 Manchester Grand Hyatt, San Diego, CA',
  'https://www.ntca.org/events-education/events/rural-telecom-industry-meet

In [80]:
ntca_dict=ntca


ntca_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=ntca_dict)
ntca_df.to_csv("ntca.tsv", sep = '\t',index=False)
print('Dusted')

Dusted


In [12]:
# driver.switch_to.window(driver.window_handles[0])

In [19]:
# ebb=[' '.join(a.text.split()) for a in m.find('p').next_siblings]
# ebb


# rim=[a for a in [' '.join(a.text.split()) for a in m.find('p').next_siblings] if len(a)>1][0]
# rim

['',
 'Next event: February 19–22, 2023 — Manchester Grand Hyatt | San Diego, CA',
 '']

In [13]:
# truce=[]
# for k in range(1,22):
#     driver.switch_to.window(driver.window_handles[k])
#     time.sleep(1)
#     soupn=bs(driver.page_source,'lxml')
    
#     try:
#         info=' '.join(soupn.find('div',{'class':'my-6 text-lg leading-relaxed'}).text.split())
#     except:
#         info=' '.join(soupn.find('div',{'class':'wysiwyg-content'}).findChild().text.split())
#         if ':' in info:
#             info='.'.join(info.split('.')[:-1])+'.'
#         if 'Click here' in info:
#             info=info.split('Click here')[0].strip()
#     #**************************
#     try:
#         mam=' '.join(soupn.find('div',{'class':'my-12'}).get_text(separator='}').split())
#         tida=mam.split('} }')[0].replace('–','-')
#         strt,enrt=tida.split('}')
#         das,tis=strt.split('|')
#         dae,tie=enrt.split('|')
#         da=das+'-'+dae
#         ti=tis+'-'+tie
#     except:
#         sham=soupn.find('div',{'class':'filterable-list--simple-two'}).text
#         diy=soupn.find_all('div',{'class':'filterable-list__row'})
#         if 'Time:' in sham:
#             for o in diy:
#                 if 'Time:' in str(o):
#                     ti=' '.join(o.find_all('div',{'class':'filterable-list__cell'})[1].text.replace('–','-').split()).strip()
#                     if '|' in ti:
#                         try:
#                             ti=ti.split('|')[0].split('):')[1].strip()
#                         except:
#                             ti=ti.split('|')[0].strip()
#                 elif 'Date:' in str(o):
#                     da=' '.join(o.find_all('div',{'class':'filterable-list__cell'})[1].text.replace('–','-').split()).strip()
#                 #tida=ti+'|'+da
#         else:
#             for o in diy:
#                 if 'Date:' in str(o):
#                     tida=' '.join(o.find_all('div',{'class':'filterable-list__cell'})[1].text.replace('–','-').split())
            
#             if '|' in tida:

#                 ii=tida.split('|')
#                 da=ii[0].strip()
#                 if '-' in da:
        
#                     da=da.split('-')[0].strip()
        
#                 if ';' in ii[1]:
#                     ti=ii[1].split(';')[0].strip()
#                 else:
#                     ti=ii[1].split(',')[0].strip()
#             else:
#                 ii=tida.split('-')
#                 da=ii[0].strip()
#                 if ';' in ii[1]:
#                     ti=ii[1].split(';')[0].strip()
#                 else:
#                     ti=ii[1].split(',')[0].strip()
#     for f in ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']:
#         if f in da:
#             da=da.replace(f,'').replace(',','').strip()
#     dat=da.replace(',','').replace('.','').replace('  ',' ').replace('Sept','September').strip()
#     if '-' in dat:
#         st=dat.split('-')[0].strip()
#         en=dat.split('-')[1].strip()
#     elif '/' in dat:
#         dg=dat.split(' ')
#         st,en=dg[0],dg[-1]
#     else:
#         st=en=dat
#     #change ti format
#     ti=ti.replace('.','').upper().replace(' AM','AM').replace(' PM','PM')

#     #
#     if '-' in ti:
#         ra=ti.split('-')[0].strip()
#         ma=ti.split('-')[1].strip()
#         try:
#             z=ma.split(' ')[1]
#             da=ma.split(' ')[0]
#         except:
#             z=''
#             da=ma
#     else:
#         ra=ti.split(' ')[0].strip()
#         da=''
#         z=ti.split(' ')[1].strip()
# #     if 'AM' in ra or 'PM' in ra:
# #         rt=datetime.strptime(da, '%H:%M')
# #         da=rt.strftime('%I:%M%p')
# #         mt=datetime.strptime(ra, '%H:%M')
# #         ra=mt.strftime('%I:%M%p')
#     time_st={'type':'general',
#              'Start_time':ra,
#              'end_time':da,
#              'timezone':z,
#              'days':'all'

#     }
#     time_=json.dumps([time_st])
#     time_
    
    
    
#     #                        02        April      2022
#     if re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
#      #   print('awujale')
#         start=st
#         end=en
#     #special case 5/26
#     elif '/' in st:
#         if len(st.split('/'))==2:
#             yr=str(datetime.today().year)
#             br=st+' '+yr
#             mo=datetime.strptime(br, '%m/%d %Y')
#             start=end=str(mo.strftime('%d %B %Y'))
#             if '/' not in en:  
#                 start=end
#             else:
#                 tr=en+' '+yr
#                 mo=datetime.strptime(tr, '%m/%d %Y')
#                 end=str(mo.strftime('%d %B %Y'))
        

#     #*                        April       02      2022
#     elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
#     #       print('ala')
#         pa=re.search('(\d{4})',st).group()
#         sa=re.search('([A-Sa-z]{3,9})',st).group()
#         ta=re.search('(\d{1,2})',st).group()
#         start=ta+' '+sa+' '+pa
#         #
#         ba=re.search('(\d{4})',en).group()
#         ca=re.search('([A-Sa-z]{3,9})',en).group()
#         da=re.search('(\d{1,2})',en).group()
#         end=da+' '+ca+' '+ba     

#     #                      02     2022
#     elif re.search('\d{1,2}\s+\d{4}',en):
#     #      print('kum')
#         pa=re.search('(\d{4})',en).group()#, maxsplit=0)
#         sa=re.search('([A-Sa-z]{3,9})',st).group()
#         ta=re.search('(\d{1,2})',st).group()

#         start=ta+' '+sa+' '+pa
#         #
#         ba=re.search('(\d{4})',en).group()
#         ca=re.search('([A-Sa-z]{3,9})',st).group()
#         da=re.search('(\d{1,2})',en).group()
#         end=da+' '+ca+' '+ba

#     #                      02         April
#     elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
#     #        print('is')
#         pa=re.search('(\d{4})',en)#, maxsplit=0)
#         sapa=pa.group()
#         start=st+' '+sapa
#         end=en

#     #*                      April           02
#     elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
#     #      print('bad')
#         try:
#             pa=re.search('(\d{4})',en).group()#, maxsplit=0)
#         except:
#             pa=str(datetime.today().year)
#         sa=re.search('([A-Sa-z]{3,9})',st).group()
#         ta=re.search('(\d{1,2})',st).group()

#         start=ta+' '+sa+' '+pa
#         #
#         try:
#             ba=re.search('(\d{4})',en).group()
#         except:
#             ba=str(datetime.today().year)
#         ca=re.search('([A-Sa-z]{3,9})',en).group()
#         da=re.search('(\d{1,2})',en).group()
#         end=da+' '+ca+' '+ba
#     #**
#     if start=='':
#         start_date=end_date=''
#     else:
#         pick=[start, end]
#         try:
#             spl_dt_obj = [datetime.strptime(v, '%d %b %Y') for v in pick]
#         except:
#             spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
#         date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
#         start_date=date_[0]
#         end_date=date_[1]

#     truce.append([time_,start_date,end_date,info])
# truce